# Day 5

## Part 1

We want to restore the Thermal Environment Supervision Terminal (Test)

First we will need the code from day 2 to execute a program, with two more instructions:
* opcode 3 read input and put it a the position given by its only parameter
* opcode 4 write to output the value a the position given by its only parameter

Secondly, parameter can have a mode :

> Each parameter of an instruction is handled based on its parameter mode. Right now, your ship computer already understands parameter mode 0, position mode, which causes the parameter to be interpreted as a position - if the parameter is 50, its value is the value stored at address 50 in memory. Until now, all parameters have been in position mode.

> Now, your ship computer will also need to handle parameters in mode 1, immediate mode. In immediate mode, a parameter is interpreted as a value - if the parameter is 50, its value is simply 50.

> Parameter modes are stored in the same value as the instruction's opcode. The opcode is a two-digit number based only on the ones and tens digit of the value, that is, the opcode is the rightmost two digits of the first value in an instruction. Parameter modes are single digits, one per parameter, read right-to-left from the opcode: the first parameter's mode is in the hundreds digit, the second parameter's mode is in the thousands digit, the third parameter's mode is in the ten-thousands digit, and so on. Any missing modes are 0.

In [1]:
from collections import defaultdict


def log(*args):
    if DEBUG:
        print(*args)


def get_param_value(program, position, param, mode):
    if mode == 0:  # position mode
        return program[program[position + param]]
    elif mode == 1:  # immediate mode
        return program[position + param]
    else:
        raise ValueError("Unknown paremeter mode")

        
def compute(program, read_input, write_output):
    """Compute the final state of a program, and return it.
    
    `program` is a list of int.
    `read_input` is a method returning a value each time it is called.
    `write_output` is a method taking a value as parameter.
    """
    program = program.copy()
    
    # init state
    p = 0
    
    # run program
    while True:
        # read instruction and split it into opcode and parameters mode
        
        try:
            instruction = program[p]
        except IndexError:
            # end of program
            break

        parameters_mode = defaultdict(lambda: 0)
        opcode = instruction % 100

        value = instruction // 100
        i = 1
        while value > 0:
            parameters_mode[i] = value % 10
            value //= 10
            i += 1
            
        log(p, '>', instruction, opcode, dict(parameters_mode))

        # execute opcode
        if opcode == 1:
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]

            result = x + y
            program[result_pos] = result
            log(x, y, result_pos, result)

            step = 4
        elif opcode == 2:
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]
            
            result = x * y
            program[result_pos] = result
            log(x, y, result_pos, result)

            step = 4
        elif opcode == 3:
            result_pos = program[p + 1]

            value = read_input()
            program[result_pos] = value
            log(value, result_pos)

            step = 2
        elif opcode == 4:
            value = get_param_value(program, p, 1, parameters_mode[1])
            write_output(value)
            log(value)
            
            step = 2
        elif opcode == 99:
            # end of program
            break

        p += step

    return program

### Test cases

In [2]:
program = [3,0,4,0,99]

result = None
def read_input():
    return 42

def write_output(value):
    global result
    result = value
    
DEBUG = True
compute(program, read_input, write_output)
assert result == 42, result

0 > 3 3 {}
42 0
2 > 4 4 {}
42
4 > 99 99 {}


In [3]:
program = [1002,4,3,4,33]
result = compute(program, read_input, write_output)

0 > 1002 2 {1: 0, 2: 1}
33 3 4 99
4 > 99 99 {}


It should do:
* 02: multiplication
  * 4 with mode 0 means take the value at pose 4: 33
  * 3 with mode 1 means take 1
  * `33 * 3 = 99`
  * put it pos 4
* move to pos 4
* 99: stop

In [4]:
result

[1002, 4, 3, 4, 99]

### Actual case

In [5]:
input_value = "3,225,1,225,6,6,1100,1,238,225,104,0,1102,67,92,225,1101,14,84,225,1002,217,69,224,101,-5175,224,224,4,224,102,8,223,223,101,2,224,224,1,224,223,223,1,214,95,224,101,-127,224,224,4,224,102,8,223,223,101,3,224,224,1,223,224,223,1101,8,41,225,2,17,91,224,1001,224,-518,224,4,224,1002,223,8,223,101,2,224,224,1,223,224,223,1101,37,27,225,1101,61,11,225,101,44,66,224,101,-85,224,224,4,224,1002,223,8,223,101,6,224,224,1,224,223,223,1102,7,32,224,101,-224,224,224,4,224,102,8,223,223,1001,224,6,224,1,224,223,223,1001,14,82,224,101,-174,224,224,4,224,102,8,223,223,101,7,224,224,1,223,224,223,102,65,210,224,101,-5525,224,224,4,224,102,8,223,223,101,3,224,224,1,224,223,223,1101,81,9,224,101,-90,224,224,4,224,102,8,223,223,1001,224,3,224,1,224,223,223,1101,71,85,225,1102,61,66,225,1102,75,53,225,4,223,99,0,0,0,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,1,99999,1105,1,280,1105,1,99999,1,225,225,225,1101,294,0,0,105,1,0,1105,1,99999,1106,0,300,1105,1,99999,1,225,225,225,1101,314,0,0,106,0,0,1105,1,99999,8,226,226,224,102,2,223,223,1005,224,329,1001,223,1,223,1108,677,677,224,1002,223,2,223,1006,224,344,101,1,223,223,1007,226,677,224,102,2,223,223,1005,224,359,101,1,223,223,1007,677,677,224,1002,223,2,223,1006,224,374,101,1,223,223,1108,677,226,224,1002,223,2,223,1005,224,389,1001,223,1,223,108,226,677,224,102,2,223,223,1006,224,404,101,1,223,223,1108,226,677,224,102,2,223,223,1005,224,419,101,1,223,223,1008,677,677,224,102,2,223,223,1005,224,434,101,1,223,223,7,677,226,224,1002,223,2,223,1005,224,449,101,1,223,223,1008,226,226,224,102,2,223,223,1005,224,464,1001,223,1,223,107,226,677,224,1002,223,2,223,1006,224,479,1001,223,1,223,107,677,677,224,102,2,223,223,1005,224,494,1001,223,1,223,1008,226,677,224,102,2,223,223,1006,224,509,1001,223,1,223,1107,677,226,224,102,2,223,223,1005,224,524,101,1,223,223,1007,226,226,224,1002,223,2,223,1006,224,539,1001,223,1,223,107,226,226,224,102,2,223,223,1006,224,554,101,1,223,223,108,677,677,224,1002,223,2,223,1006,224,569,1001,223,1,223,7,226,677,224,102,2,223,223,1006,224,584,1001,223,1,223,8,677,226,224,102,2,223,223,1005,224,599,101,1,223,223,1107,677,677,224,1002,223,2,223,1005,224,614,101,1,223,223,8,226,677,224,102,2,223,223,1005,224,629,1001,223,1,223,7,226,226,224,1002,223,2,223,1006,224,644,1001,223,1,223,108,226,226,224,1002,223,2,223,1006,224,659,101,1,223,223,1107,226,677,224,1002,223,2,223,1006,224,674,101,1,223,223,4,223,99,226"

program = [int(x) for x in input_value.split(",")]

In [6]:
DEBUG=False

def read_input():
    # Only one value should be read
    return 1

result = []
def write_output(value):
    global result
    result.append(value)

DEBUG = False
compute(program, read_input, write_output)
result

[0, 0, 0, 0, 0, 0, 0, 0, 0, 5074395]

## Part two

We need to add several more instruction to our interperter: jump-if-true, jump-if-false, lt and eq.

In [7]:
from collections import defaultdict



def log(*args):
    if DEBUG:
        print(*args)


def get_param_value(program, position, param, mode):
    if mode == 0:  # position mode
        return program[program[position + param]]
    elif mode == 1:  # immediate mode
        return program[position + param]
    else:
        raise ValueError("Unknown paremeter mode")

        
def compute(program, read_input, write_output):
    """Compute the final state of a program, and return it.
    
    `program` is a list of int.
    `read_input` is a method returning a value each time it is called.
    `write_output` is a method taking a value as parameter.
    """
    program = program.copy()
    
    # init state
    p = 0
    
    # run program
    while True:
        # read instruction and split it into opcode and parameters mode
        
        try:
            instruction = program[p]
        except IndexError:
            # end of program
            break

        parameters_mode = defaultdict(lambda: 0)
        opcode = instruction % 100

        value = instruction // 100
        i = 1
        while value > 0:
            parameters_mode[i] = value % 10
            value //= 10
            i += 1
            
        log(p, '>', instruction, opcode, dict(parameters_mode))

        # execute opcode
        if opcode == 1:
            # add x y
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]

            result = x + y
            program[result_pos] = result
            log("add", x, y, result_pos, result)

            step = 4
        elif opcode == 2:
            # mult x y
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]
            
            result = x * y
            program[result_pos] = result
            log("mult", x, y, result_pos, result)

            step = 4
        elif opcode == 3:
            # read x
            result_pos = program[p + 1]

            value = read_input()
            program[result_pos] = value
            log("read", value, result_pos)

            step = 2
        elif opcode == 4:
            # write x
            value = get_param_value(program, p, 1, parameters_mode[1])
            write_output(value)
            log("write", value)
            
            step = 2
        elif opcode == 5:
            # jump-if-true x y
            value = get_param_value(program, p, 1, parameters_mode[1])
            jump_pos = get_param_value(program, p, 2, parameters_mode[2])
            
            log("jump-if-true", value, jump_pos)
            
            if value:
                p = jump_pos
                step = 0
            else:
                step = 3
        elif opcode == 6:
            # jump-if-false x y
            value = get_param_value(program, p, 1, parameters_mode[1])
            jump_pos = get_param_value(program, p, 2, parameters_mode[2])
            
            log("jump-if-false", value, jump_pos)
            
            if not value:
                p = jump_pos
                step = 0
            else:
                step = 3
        elif opcode == 7:
            # lt x y z
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]
            
            log("lt", x, y, result_pos)
            
            if x < y:
                program[result_pos] = 1
            else:
                program[result_pos] = 0
                
            step = 4
        elif opcode == 8:
            # eq x y z
            x = get_param_value(program, p, 1, parameters_mode[1])
            y = get_param_value(program, p, 2, parameters_mode[2])
            result_pos = program[p + 3]
            
            log("eq", x, y, result_pos)
            
            if x == y:
                program[result_pos] = 1
            else:
                program[result_pos] = 0
                
            step = 4
        elif opcode == 99:
            # end of program
            log("stop")
            break
        else:
            # unknown instruction
            log("unknown opcode")
            break

        p += step

    return program

### Test cases

Read input and output 1 or 0 if it is equal to 8.

In [8]:
DEBUG = False
program = [3,9,8,9,10,9,4,9,99,-1,8]

result = []
read_input = lambda: 1
compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 8
compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 10
compute(program, read_input, write_output)
print(result)

[0]
[1]
[0]


Read the input and output 1 or 0 if it is less than 8

In [9]:
DEBUG=False
program = [3,9,7,9,10,9,4,9,99,-1,8]

result = []
read_input = lambda: 1
compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 8
compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 10
compute(program, read_input, write_output)
print(result) 

[1]
[0]
[0]


Read the input and output 1 or 0 if it is equal to 8, with immediate mode.

In [10]:
DEBUG=False
program = [3,3,1108,-1,8,3,4,3,99]

result = []
read_input = lambda: 1
compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 8
program = compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 10
compute(program, read_input, write_output)
print(result)

[0]
[1]
[0]


Read the input and output 1 or 0 if it is less than 8, with immediate mode.

In [11]:
DEBUG=False
program = [3,3,1107,-1,8,3,4,3,99]

result = []
read_input = lambda: 1
compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 8
program = compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 10
compute(program, read_input, write_output)
print(result)

[1]
[0]
[0]


Take an input, then output 0 if the input was zero or 1 if the input was non-zero.

In [12]:
DEBUG = False

# position mode

program = [3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9]

result = []
read_input = lambda: 0
compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 3
compute(program, read_input, write_output)
print(result)

# immediate mode

program = [3,3,1105,-1,9,1101,0,0,12,4,12,99,1]

result = []
read_input = lambda: 0
compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 3
compute(program, read_input, write_output)
print(result)

[0]
[1]
[0]
[1]


The program uses an input instruction to ask for a single number. The program will then output 999 if the input value is below 8, output 1000 if the input value is equal to 8, or output 1001 if the input value is greater than 8.

In [13]:
DEBUG = False

# position mode

program = [
    3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
    1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
    999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99
]

result = []
read_input = lambda: 0
compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 8
compute(program, read_input, write_output)
print(result)

result = []
read_input = lambda: 12
compute(program, read_input, write_output)
print(result)

[999]
[1000]
[1001]


### Actual case

In [14]:
program = [3,225,1,225,6,6,1100,1,238,225,104,0,1102,67,92,225,1101,14,84,225,1002,217,69,224,101,-5175,224,224,4,224,102,8,223,223,101,2,224,224,1,224,223,223,1,214,95,224,101,-127,224,224,4,224,102,8,223,223,101,3,224,224,1,223,224,223,1101,8,41,225,2,17,91,224,1001,224,-518,224,4,224,1002,223,8,223,101,2,224,224,1,223,224,223,1101,37,27,225,1101,61,11,225,101,44,66,224,101,-85,224,224,4,224,1002,223,8,223,101,6,224,224,1,224,223,223,1102,7,32,224,101,-224,224,224,4,224,102,8,223,223,1001,224,6,224,1,224,223,223,1001,14,82,224,101,-174,224,224,4,224,102,8,223,223,101,7,224,224,1,223,224,223,102,65,210,224,101,-5525,224,224,4,224,102,8,223,223,101,3,224,224,1,224,223,223,1101,81,9,224,101,-90,224,224,4,224,102,8,223,223,1001,224,3,224,1,224,223,223,1101,71,85,225,1102,61,66,225,1102,75,53,225,4,223,99,0,0,0,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,1,99999,1105,1,280,1105,1,99999,1,225,225,225,1101,294,0,0,105,1,0,1105,1,99999,1106,0,300,1105,1,99999,1,225,225,225,1101,314,0,0,106,0,0,1105,1,99999,8,226,226,224,102,2,223,223,1005,224,329,1001,223,1,223,1108,677,677,224,1002,223,2,223,1006,224,344,101,1,223,223,1007,226,677,224,102,2,223,223,1005,224,359,101,1,223,223,1007,677,677,224,1002,223,2,223,1006,224,374,101,1,223,223,1108,677,226,224,1002,223,2,223,1005,224,389,1001,223,1,223,108,226,677,224,102,2,223,223,1006,224,404,101,1,223,223,1108,226,677,224,102,2,223,223,1005,224,419,101,1,223,223,1008,677,677,224,102,2,223,223,1005,224,434,101,1,223,223,7,677,226,224,1002,223,2,223,1005,224,449,101,1,223,223,1008,226,226,224,102,2,223,223,1005,224,464,1001,223,1,223,107,226,677,224,1002,223,2,223,1006,224,479,1001,223,1,223,107,677,677,224,102,2,223,223,1005,224,494,1001,223,1,223,1008,226,677,224,102,2,223,223,1006,224,509,1001,223,1,223,1107,677,226,224,102,2,223,223,1005,224,524,101,1,223,223,1007,226,226,224,1002,223,2,223,1006,224,539,1001,223,1,223,107,226,226,224,102,2,223,223,1006,224,554,101,1,223,223,108,677,677,224,1002,223,2,223,1006,224,569,1001,223,1,223,7,226,677,224,102,2,223,223,1006,224,584,1001,223,1,223,8,677,226,224,102,2,223,223,1005,224,599,101,1,223,223,1107,677,677,224,1002,223,2,223,1005,224,614,101,1,223,223,8,226,677,224,102,2,223,223,1005,224,629,1001,223,1,223,7,226,226,224,1002,223,2,223,1006,224,644,1001,223,1,223,108,226,226,224,1002,223,2,223,1006,224,659,101,1,223,223,1107,226,677,224,1002,223,2,223,1006,224,674,101,1,223,223,4,223,99,226]

DEBUG=False
result = []
read_input = lambda: 5
compute(program, read_input, write_output)
result

[8346937]